In [ ]:
# Codigo para Media dos Mais Caros e Mais Baratos ANP em SP e RJ
# Realizado: Media ponderada para cada combustivel de cada estado

comb = ['diesel_rj', 'diesel_sp', 'diesel_s10_rj', 'diesel_s10_sp', 
        'etanol_rj', 'etanol_sp', 'gasol_adit_rj', 'gasol_adit_sp', 
        'gasolina_sp', 'gasolina_rj', 'glp_rj', 'glp_sp', 'gnv_rj', 'gnv_sp']

data = {'Diesel': [], 'Diesel S10': [], 'Etanol': [], 'Gasolina Adit': [], 'Gasolina': [], 'GLP': [], 'GNV': []}
data_keys = list(data.keys())

url = 'https://www.primetalkdata.com/datacenter/combustiveis/'

for type in ['mais_caro/', 'mais_barato/']:
    date = requests.get(url + type).text.split('<a href="')[-1][:10]
    
    for idx in range(len(comb)):
        auxDF = pd.DataFrame([elem.split(',') for elem in requests.get(url + type + date + '_' + comb[idx] + '.csv').text.split('\r\n')])
        auxDF.columns = auxDF.iloc[0]
        auxDF.drop(auxDF.index[0], inplace=True)
        auxDF.dropna(inplace=True)

        val = auxDF[auxDF['DATA INICIAL'] == auxDF['DATA INICIAL'].iloc[-1]].iloc[:,-1:].astype(float).values
        wgt = auxDF[auxDF['DATA INICIAL'] == auxDF['DATA INICIAL'].iloc[-1]].iloc[:,-3:-2].astype(int).values

        data[data_keys[idx//2]].append(round(np.average(val, weights=wgt), 2))

pd.DataFrame(data, index=pd.MultiIndex.from_tuples([('RJ', 'Mais Caro'), ('RJ', 'Mais Barato'), ('SP', 'Mais Caro'), ('SP', 'Mais Barato')]))

In [ ]:
# Codigo para 5 Maiores Altas e 5 Maiores Quedas pela CEAGESP
# Realizado: 5 maiores altas e baixas percentuais entre todos os setores 

url = 'https://www.primetalkdata.com/datacenter/ceagesp/2023/'
date = requests.get(url).text.split('<a href="')[-1][:10]

data = {'Tipo': [], 'Setor': [], 'Variação': [], 'Item': [], 'Percentual': []}

for type in ['alta', 'baixa']:
    resp = requests.get(url + date + '_ceagesp.csv')
    resp.encoding = resp.apparent_encoding
    auxDF = pd.DataFrame([elem.split(',') for elem in resp.text.split('\r\n')])
    auxDF.columns = auxDF.iloc[0]
    auxDF.drop(auxDF.index[0], inplace=True)
    auxDF.dropna(inplace=True)

    auxDF['pct_' + type] = pd.to_numeric(auxDF['pct_' + type], errors='coerce')
    auxDF = auxDF.sort_values('pct_' + type, ascending=(type=='baixa')).head(5)
    
    data['Tipo'].extend(['Maiores Altas']*5 if type == 'alta' else ['Maiores Baixas']*5) 
    data['Setor'].extend(auxDF.setor.values)
    data['Variação'].extend(auxDF.variacao.values)
    data['Item'].extend(auxDF['top_' + type + 's'].values)
    data['Percentual'].extend(auxDF['pct_' + type].values) 

pd.DataFrame(data).set_index(['Tipo', 'Setor'])